# GDI query

Let's now execute some queries on this data. First we'll connect to the project.

In [1]:
from pathlib import Path

import genomics_data_index.api as gdi

project_dir = Path('data/sars-cov-2-genbank-index')

db = gdi.GenomicsDataIndex.connect(project_dir)
db

/home/CSCScience.ca/apetkau/.local/lib/python3.8/site-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.4) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


<GenomicsDataIndex(samples=395)>

# Load up metadata

In [5]:
import pandas as pd

# metadata_df = pd.read_csv(project_dir / 'metadata.tsv', dtype=str)
# metadata_df

NameError: name 'metadata_df' is not defined

# Start query

In [7]:
reference_name = db.reference_names()[0]
reference_name

'MN996528.1-ref'

In [10]:
t = db.samples_query(universe='mutations_experimental', reference_name=reference_name).set_outgroup(reference_name)
t

<ExperimentalTreeSamplesQuery[100% (395/395) samples]>

# Define mutations as amino acid names

I currently don't have nucleotide to amino acid mutation mapping working, so I have to do that here myself in a dictionary.

In [19]:
t.summary_features().sort_values('Count')

,Sequence,Position,Deletion,Insertion,Count
Mutation,,,,,
MN996528.1:16690:G:T,MN996528.1,16690,G,T,1
MN996528.1:14755:C:T,MN996528.1,14755,C,T,1
MN996528.1:14747:A:G,MN996528.1,14747,A,G,1
MN996528.1:26568:C:A,MN996528.1,26568,C,A,1
MN996528.1:14659:T:C,MN996528.1,14659,T,C,1
...,...,...,...,...,...
MN996528.1:15324:C:T,MN996528.1,15324,C,T,135
MN996528.1:3037:C:T,MN996528.1,3037,C,T,361
MN996528.1:241:C:T,MN996528.1,241,C,T,364


In [15]:
mut = {
    
}

# Visualize tree

In [35]:
# t.tree_styler(mode='c', show_leaf_names=False,
#               annotate_box_width=100, annotate_opacity_absent=1.0, annotate_border_width=1,
#               annotate_guiding_lines=False)\
#     .annotate(t.hasa('MN996528.1:15324:C:T'))\
#     .render(w=500)